In [25]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os
from datetime import datetime, timedelta, timezone
import time
import uuid
import random
import pandas as pd

In [22]:
unix_timestamp = time.time()
print(unix_timestamp)

1726742259.991108


In [2]:
# Set the environment variable to use the local Firestore emulator
os.environ["FIRESTORE_EMULATOR_HOST"] = "host.docker.internal:8081"

In [3]:
cred = credentials.Certificate("./_serviceAccount.json")
firebase_admin.initialize_app(cred)

In [4]:
db = firestore.client()

In [8]:
# Execute the query and get the results
wht_collection = db.collection('whtCodes')

In [12]:
# Execute the query and get the results
doc_ref = wht_collection.document('SG')
doc_ref.set({
    "whtCodes": [{
        "code": "1A",
        "isImportant": True,
        "name": "WC110",
        "perc": 5,
        "sourceId": "1"
    }, {
        "code": "2A",
        "name": "WC117",
        "perc": 2,
        "sourceId": "2"
    }, {
        "code": "WHT15",
        "isImportant": True,
        "name": "WHT15%",
        "perc": 15,
        "sourceId": "3"
    }, {
        "code": "WHT10",
        "isImportant": True,
        "name": "WHT10%",
        "perc": 10,
        "sourceId": "4"
    }]
})

update_time {
  seconds: 1725797609
  nanos: 192093000
}

In [23]:
def generate_sample_bill(bill_number):
    """Generate a sample bill with the given bill number."""
    return {
        "action": "create",
        "billId": str(uuid.uuid4()),
        "vendorId": "wFW2BNhegaFQBM0TTboR",
        "vendorName": "Test Vendor From NS Intg 4",
        "vendorAccountNumber": f"ACC-{random.randint(10000, 99999)}",
        "billStatus": random.choice(["draft", "scheduled", "paid"]),
        "billNumber": bill_number,
        "lineItems": [
            {
                "name": f"Item {i}",
                "quantity": random.randint(1, 10),
                "unitPrice": round(random.uniform(10, 1000), 2),
                "account": "Bank Fees",
                "accountId": "fEhhLF52zpNMznntWhi1",
                "type": "expense",
                "wht": {
                    "referenceId": f"WHT-{random.randint(1000, 9999)}",
                    "code": "1A",
                    "displayName": "WC110",
                    # "perc": random.randint(1, 10)
                    "perc": 11
                }
            } for i in range(random.randint(1, 5))
        ],
        "total": 0,  # We'll calculate this based on line items
        "subTotal": 0,  # We'll calculate this based on line items
        "paymentAmount": 0,  # We'll calculate this based on total
        "date": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
        "dueDate": (datetime.now() + timedelta(days=random.randint(1, 30))).strftime("%Y-%m-%d"),
        "currency": 'SGD',
        "billCurrency": 'SGD',
        "currencyRate": round(random.uniform(0.8, 1.2), 4),
        "isVendorSynced": True,
        "peakfloId": f"PF-{random.randint(10000, 99999)}",
        "billCreatedAt": datetime.now().isoformat(),
        "type": "BILL",
        "status": "ready",
        "customerId": "demo1Company",
        "updatedAt": firestore.SERVER_TIMESTAMP
    }

def calculate_bill_totals(bill):
    """Calculate totals for the bill based on line items."""
    sub_total = 0
    total_wht = 0
    
    for item in bill["lineItems"]:
        quantity = item["quantity"]
        unit_price = item["unitPrice"]
        item_amount = round(quantity * unit_price, 2)
        item["amount"] = item_amount
        sub_total += item_amount
        
        if "wht" in item:
            wht_perc = item["wht"]["perc"]
            wht_amount = round(item_amount * (wht_perc / 100), 2)
            item["wht"]["amount"] = wht_amount
            total_wht += wht_amount
    
    bill["subTotal"] = round(sub_total, 2)
    bill["totalWHT"] = round(total_wht, 2)
    bill["total"] = round(sub_total - total_wht, 2)
    bill["paymentAmount"] = bill["total"]

def insert_bills_to_firestore(num_bills=10):
    """Insert the specified number of bills into Firestore."""
    collection_ref = db.collection("netsuite-bill-outgoing-updates")
    
    for i in range(1, num_bills + 1):
        bill_number = f"BILL-{time.time()}-{i:03d}"
        bill = generate_sample_bill(bill_number)
        calculate_bill_totals(bill)
        
        # add bill
        collection_ref.add(bill)
        
        print(f"Inserted bill with number: {bill_number}")

In [61]:
insert_bills_to_firestore(2)

Inserted bill with number: BILL-1726743713.0499072-001
Inserted bill with number: BILL-1726743713.527992-002


In [62]:
# Reference to your collection
collection_ref = db.collection('netsuite-bill-outgoing-updates')

# Fetch all documents
docs = collection_ref.get()

# Convert to list of dictionaries, including the document ID
data = []
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['document_id'] = doc.id  # Add the document ID to the dictionary
    data.append(doc_dict)

# Create pandas DataFrame
df = pd.DataFrame(data)

# Reorder columns to have document_id as the first column
cols = ['document_id'] + [col for col in df.columns if col != 'document_id']
df = df[cols]

# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to multiple lines
pd.set_option('max_colwidth', None)  # Show full contents of each cell

In [63]:
df

,document_id,billNumber,vendorId,type,sourceId,failureReason,status,updatedAt,notes,dueDate,date,customerId,currency,lineItems,billId,action,paymentAmount,currencyRate,totalWHT,vendorName,subTotal,billCreatedAt,vendorAccountNumber,total,peakfloId,billCurrency,billStatus,isVendorSynced
0,46LrcyPYJb2KYvV1Imdc,Test Bill From Local 3750289395222,wFW2BNhegaFQBM0TTboR,BILL,61346.0,,completed,2024-08-23 21:00:06.612000+00:00,Test Bill From Local 3750289395222,2024-08-30T07:14:24+00:00,2024-08-23T07:14:24+00:00,demo1Company,SGD,"[{'accountId': 'Ovgtbw5GVWPeM0tzm3VU', 'total': 126, 'name': 'test place holder line item name'}]",pw7rgLTgaYYHAK7fkh4PeIH6J8ACsm,create,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8EgXmoDpoCnov4w3huLg,BILL-1726742353.6395614-001,wFW2BNhegaFQBM0TTboR,BILL,66943.0,,completed,2024-09-19 10:44:13.407000+00:00,NaN,2024-09-22,2024-09-12,demo1Company,SGD,"[{'quantity': 4, 'unitPrice': 544.84, 'type': 'expense', 'account': 'Bank Fees', 'accountId': 'fEhhLF52zpNMznntWhi1', 'wht': {'referenceId': 'WHT-2471', 'displayName': 'WC110', 'perc': 11, 'amount': 239.73, 'code': '1A'}, 'amount': 2179.36, 'name': 'Item 0'}, {'quantity': 6, 'unitPrice': 425.17, 'type': 'expense', 'account': 'Bank Fees', 'accountId': 'fEhhLF52zpNMznntWhi1', 'wht': {'referenceId': 'WHT-4724', 'displayName': 'WC110', 'perc': 11, 'amount': 280.61, 'code': '1A'}, 'amount': 2551.02, 'name': 'Item 1'}, {'quantity': 1, 'unitPrice': 786.05, 'type': 'expense', 'account': 'Bank Fees', 'accountId': 'fEhhLF52zpNMznntWhi1', 'wht': {'referenceId': 'WHT-6302', 'displayName': 'WC110', 'perc': 11, 'amount': 86.47, 'code': '1A'}, 'amount': 786.05, 'name': 'Item 2'}]",5209b7fd-6bb0-402b-948b-2d494df152fe,create,4909.62,0.8228,606.81,Test Vendor From NS Intg 4,5516.43,2024-09-19T10:39:13.670895,ACC-77152,4909.62,PF-58272,SGD,scheduled,True
2,9gCEcCjzwqXsCm1yyya7,Test Bill From Local 8358965890772,wFW2BNhegaFQBM0TTboR,BILL,61341.0,,completed,2024-08-23 21:00:06.596000+00:00,Test Bill From Local 8358965890772,2024-08-30T07:14:24+00:00,2024-08-23T07:14:24+00:00,demo1Company,SGD,"[{'accountId': 'Ovgtbw5GVWPeM0tzm3VU', 'total': 121, 'name': 'test place holder line item name'}]",XNP6FgzTJ1F3lIeofB5IC9PjKyiHap,create,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BmunpbC5ib04AaIXkddq,Test Bill From Local 4303450306964,wFW2BNhegaFQBM0TTboR,BILL,61344.0,,completed,2024-08-23 21:00:06.602000+00:00,Test Bill From Local 4303450306964,2024-08-30T07:14:24+00:00,2024-08-23T07:14:24+00:00,demo1Company,SGD,"[{'accountId': 'Ovgtbw5GVWPeM0tzm3VU', 'total': 124, 'name': 'test place holder line item name'}]",tGghN4CDVieDcZe8QXqLIjOXXokK6S,create,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N2kkvAsqGuLkwmMybQaF,Test Bill From Local 7802777782462,wFW2BNhegaFQBM0TTboR,BILL,61347.0,,completed,2024-08-23 21:00:06.605000+00:00,Test Bill From Local 7802777782462,2024-08-30T07:14:24+00:00,2024-08-23T07:14:24+00:00,demo1Company,SGD,"[{'accountId': 'Ovgtbw5GVWPeM0tzm3VU', 'total': 127, 'name': 'test place holder line item name'}]",9BikbUYBv1BTzWohEbIdQoxFEaUNOA,create,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sqemu4DWNbjVW8I6ini7,BILL-1726742358.607891-002,wFW2BNhegaFQBM0TTboR,BILL,66944.0,,completed,2024-09-19 10:44:13.422000+00:00,NaN,2024-09-28,2024-08-20,demo1Company,SGD,"[{'quantity': 4, 'unitPrice': 207.07, 'type': 'expense', 'account': 'Bank Fees', 'accountId': 'fEhhLF52zpNMznntWhi1', 'wht': {'referenceId': 'WHT-3609', 'displayName': 'WC110', 'perc': 11, 'amount': 91.11, 'code': '1A'}, 'amount': 828.28, 'name': 'Item 0'}, {'quantity': 7, 'unitPrice': 841.82, 'type': 'expense', 'account': 'Bank Fees', 'accountId': 'fEhhLF52zpNMznntWhi1', 'wht': {'referenceId': 'WHT-9286', 'displayName': 'WC110', 'perc': 11, 'amount': 648.2, 'code': '1A'}, 'amount': 5892.74, 'name': 'Item 1'}]",61d3df50-f85a-40b8-b9f6-624f54b5b902,create,5981.71,0.9519,739.31,Test Vendor From NS Intg 4,6721.02,2024-09-19T10:39:18.608113,ACC-16789,5981.71,PF-50276,SGD,draft,True
6,ZkIVmPjjiRRqZic5T5Sv,Test